<a href="https://colab.research.google.com/github/tugaevaea/Web_Mining/blob/master/Pre_trained_BERT_NER_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# feed the whole text into pretraind hugging face. model ner 
# the next step we need (word,label) tuples to assess labels f1 score or accuracy of pretrained model 

In [0]:
import pandas as pd
import numpy as np
import io
from google.colab import files
uploaded = files.upload()
data = pd.read_csv(io.StringIO(uploaded['ner_dataset.csv'].decode('latin1'))).fillna(method="ffill")
data.tail(100)

Saving ner_dataset.csv to ner_dataset (1).csv


,Sentence #,Word,POS,Tag
1048475,Sentence: 47954,pushed,VBD,O
1048476,Sentence: 47954,the,DT,O
1048477,Sentence: 47954,Taleban,NNP,B-org
1048478,Sentence: 47954,from,IN,O
1048479,Sentence: 47954,power,NN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [0]:
pip install transformers

     |████████████████████████████████| 665kB 9.5MB/s 
     |████████████████████████████████| 3.8MB 24.5MB/s 
     |████████████████████████████████| 1.1MB 54.0MB/s 
     |████████████████████████████████| 890kB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=094ae0051804c370cc15a632757dfff1f50534647836a06096f0a2df18cb4410
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)


'Tesla P4'

In [0]:
# approximately 100K were taken
data1 = data.loc[0:102399, :]
data1.tail()

,Sentence #,Word,POS,Tag
102395,Sentence: 4653,the,DT,O
102396,Sentence: 4653,men,NNS,O
102397,Sentence: 4653,were,VBD,O
102398,Sentence: 4653,planning,VBG,O
102399,Sentence: 4653,.,.,O


In [0]:
initial_word_tag_tuple_list = list(zip(data1["Word"].values.tolist(), data1["Tag"].values.tolist()))
len(initial_word_tag_tuple_list)

In [0]:
text = ' '.join(word for word in data1['Word'] )
text

In [0]:
pip install seqeval

In [0]:
# BERT-NER model 
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",       # Outside of a named entity
    "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
    "I-MISC",  # Miscellaneous entity
    "B-PER",   # Beginning of a person's name right after another person's name
    "I-PER",   # Person's name
    "B-ORG",   # Beginning of an organisation right after another organisation
    "I-ORG",   # Organisation
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC"    # Location
]



tokens = tokenizer.tokenize(text)


#data splitted in chunks as the model can process limited number of tokens per iteration

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
t = 0
word_pred = []
l = chunks(list(tokens),54)
for i in l:
  inputs = tokenizer.encode(i, return_tensors="pt")
  outputs = model(inputs)[0]
  predictions = torch.argmax(outputs, dim=2)
  for j, k in zip(i,predictions[0][1:].tolist()):
    word_pred.append((j,label_list[k-1]))
    t +=1
  
print(word_pred)

In [0]:
word_pred = [(x[0],x[1].lower()) for x in word_pred]
initial_word_tag_tuple_list = [(x[0],x[1].lower()) for x in initial_word_tag_tuple_list ]

In [0]:
print(len(word_pred))
print(len(initial_word_tag_tuple_list))



118046
102400


In [0]:
# to get alligned lists of predictions and true values 
a = initial_word_tag_tuple_list
b = word_pred
y_true, y_pred = [], []
sentence = [x[0] for x in b]
for x in range(len(a)):
    if a[x][0] in sentence:
        y_true.append(a[x][1].lower())
        y_pred.append(b[sentence.index(a[x][0])][1].lower())
        sentence[x] = 'AnyRandomString'
print(len(y_true))
print(len(y_pred))



93586
93586


In [0]:
print(y_pred)

['i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-misc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-per', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-loc', 'b-loc', 'b-org', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-loc', 'b-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-loc', 'i-loc', 'b-org', 'b-org', 'i-loc', 'i-loc', 'i-loc', 'b-misc', 'i-loc', 'i-loc', 'b-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'i-loc', 'b-loc', 'i-loc', 'i-loc', 'i-loc', 'b-loc', 'i-loc', 'i-loc', 'i-loc'

In [0]:
print(set(y_true))
print(set(y_pred))



{'i-tim', 'b-geo', 'o', 'i-per', 'i-org', 'i-art', 'b-per', 'b-tim', 'b-nat', 'i-nat', 'i-geo', 'b-gpe', 'i-gpe', 'b-org', 'b-eve', 'i-eve', 'b-art'}
{'b-misc', 'o', 'i-loc', 'b-per', 'b-org', 'b-loc'}


In [0]:
n = []

for x in y_pred:
    if x == 'i-loc':
      new_string = x.replace("i-loc", "o")
      n.append(new_string)
    else:
      n.append(x)  

y_pred = n
print(y_pred)

['o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'b-misc', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-per', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'b-loc', 'b-org', 'o', 'o', 'o', 'o', 'b-loc', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'b-org', 'b-org', 'o', 'o', 'o', 'b-misc', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'b-misc', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'b-loc', 'o', 'o', 'b-loc', 'o', 'o', 'b-org', 'b-org', 'b-org', 'b-org', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o',

In [0]:

# tags manipulation
new_strings = []

for x in y_true:
    if x == 'b-geo':
      new_string = x.replace("b-geo", "b-loc")
      new_strings.append(new_string)
    elif x == 'i-geo':
      new_string = x.replace("i-geo", "i-loc")
      new_strings.append(new_string)
    elif x == 'o-':
      new_string = x.replace("o-", "o")
      new_strings.append(new_string)
    elif x == 'b-gpe':
      new_string = x.replace("b-gpe", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-gpe':
      new_string = x.replace("i-gpe", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-tim':
      new_string = x.replace("b-tim", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-tim':
      new_string = x.replace("i-tim", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-art':
      new_string = x.replace("b-art", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-art':
      new_string = x.replace("i-art", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-eve':
      new_string = x.replace("b-eve", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-eve':
      new_string = x.replace("i-eve", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-nat':
      new_string = x.replace("b-nat", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-nat':
      new_string = x.replace("i-nat", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-geo':
      new_string = x.replace("b-geo", "b-loc")
      new_strings.append(new_string)
    elif x == 'i-geo':
      new_string = x.replace("i-geo", "i-loc")
      new_strings.append(new_string)
    elif x == 'b-gpe':
      new_string = x.replace("b-gpe", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-gpe':
      new_string = x.replace("i-gpe", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-tim':
      new_string = x.replace("b-tim", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-tim':
      new_string = x.replace("i-tim", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-art':
      new_string = x.replace("b-art", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-art':
      new_string = x.replace("i-art", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-eve':
      new_string = x.replace("b-eve", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-eve':
      new_string = x.replace("i-eve", "i-misc")
      new_strings.append(new_string)
    elif x == 'b-nat':
      new_string = x.replace("b-nat", "b-misc")
      new_strings.append(new_string)
    elif x == 'i-nat':
      new_string = x.replace("i-nat", "i-misc")
      new_strings.append(new_string)
    else:
      new_strings.append(x)
y_true = new_strings




In [0]:
print(set(y_true))
print(set(y_pred))
print(y_true)
print(y_pred)

{'b-misc', 'i-misc', 'o', 'i-loc', 'i-org', 'i-per', 'b-per', 'b-loc', 'b-org'}
{'b-misc', 'o', 'b-per', 'b-org', 'b-loc'}
['o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'b-misc', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-per', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'i-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'b-org', 'i-org', 'o', 'o', 'o', 'b-misc', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'b-misc', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'b-misc', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'o', 'b-loc', 'o', 'o', 'b-loc', 'o', 'o', 'b-org', 'i-org', 'i-org', 'i-org', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-loc', 'b-misc', 

In [0]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
 




print(accuracy_score(y_true, y_pred))


print(f1_score(y_true,y_pred))


print(classification_report(y_true,y_pred))


0.917380804821234
0.5610241273100617
           precision    recall  f1-score   support

        o       0.61      0.52      0.56      8154
      per       0.59      0.54      0.56      1196
      loc       0.69      0.81      0.75      2543
      org       0.38      0.40      0.39      1370
     misc       0.65      0.36      0.47      3481

micro avg       0.61      0.52      0.56     16744
macro avg       0.61      0.52      0.55     16744

